Before running this code, you need to set up the Earth Engine Python API.
Follow these steps (based on official Google documentation):
1. Install the Earth Engine API if not already installed (e.g., in a terminal: pip install earthengine-api --upgrade).
2. Create a Google Cloud Project: Go to [link](https://console.cloud.google.com/projectcreate) and create a new project. Note your Project ID (e.g., 'my-earth-engine-project').
3. Enable the Earth Engine API: In your Google Cloud Console, search for "Earth Engine API" and enable it for your project.
4. Authenticate: Run ee.Authenticate() below if prompted; this will open a browser for Google sign-in and generate a token.
5. Initialize: Replace 'your-project-id' below with your actual Google Cloud Project ID.
For more details, see: [link](https://developers.google.com/earth-engine/guides/python_install)

# Initialization

In [1]:
#import relevant libraries
import ee
import geemap

In [2]:
#Authenticate and initilaize GEE 
try:
    ee.Initialize(project='uss-dissertation')
except Exception:
    print("GEE initialization failed. Run ee.Authenticate() and try again.")
    ee.Authenticate()
    ee.Initialize(project='uss-dissertation')

# Calculations

From litrature, many studies utilize Landsat indices to assess land composition (like vegetation and impervious surface). The **Normalized Difference Vegetation Index (NDVI)** is mostly use for vegetation but there are many indices used as proxies for impervious surfaces, with various accuracy depending on climate and season. The **Normalized Difference Impervious Surface Index (NDISI)** was calculated for the study are but validation with satellite imagery showed concerning results, and i found litrature supporting its low acuracy in arid regions [Chen et al., 2019](https://www.spiedigitallibrary.org/journals/journal-of-applied-remote-sensing/volume-13/issue-1/016502/Enhanced-normalized-difference-index-for-impervious-surface-area-estimation-at/10.1117/1.JRS.13.016502.full)

After reviwing more studies, i found promising support for the **Perpendicular Impervious Surface Index (PISI)** in arid regions [Almohamad & Alshwesh, 2023](https://www.mdpi.com/2071-1050/15/12/9704#B37-sustainability-15-09704). 

NDVI and PISP are calculated from below equasions adopted from ([Li et al., 2019](https://ieeexplore-ieee-org.libproxy.ucl.ac.uk/abstract/document/9381593); [Saleem et al., 2020](https://pubmed.ncbi.nlm.nih.gov/32748362/))

 $$NDVI = \frac{NIR - Red}{NIR + Red}$$
  
  Where:
  - NIR is the near-infrared band.
  - Red is the red band.

$$PISI = 0.8192 b_{blue} − 0.5735b_{nir} + 0.075$$


  Where:
  - b~blue~ is the reflectance of the blue band.
  - b~nir~ is the reflectance of the nir band.


  **Note:** I'm using Landsat for consistency across layers at 30m resolution instead of Sentinal-2 10m resolution for NDVI.


In [3]:
# Load city boundary from a Google Earth Engine asset
def load_city_boundary(asset_id):
    try:
        city_boundary = ee.FeatureCollection(asset_id)
        print("City boundary loaded successfully!")
        return city_boundary
    except Exception as e:
        print(f"Failed to load city boundary: {str(e)}")
        return None

In [4]:
# Calculate NDVI and PISI for 2024 from Landsat for the specified year (consistency at 30m resolution).
def calculate_ndvi_pisi(city_boundary, years):
    ndvi_layers = {}
    pisi_layers = {}
    geometry = city_boundary.geometry()
    for year in years:
        # Filter for summer months to align with LST data, cloud cover <5%
        start_date = f'{year}-06-01'
        end_date = f'{year}-08-31'

        landsat = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                   .filterBounds(geometry)
                   .filterDate(start_date, end_date)
                   .filter(ee.Filter.lt('CLOUD_COVER', 5))) #cloud cover (<5%) to minimize atmospheric interference.
        landsat = landsat.merge(
            ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
            .filterBounds(geometry)
            .filterDate(start_date, end_date)
            .filter(ee.Filter.lt('CLOUD_COVER', 5)) 
        )
        landsat_size = landsat.size().getInfo()
        print(f"Number of Landsat images for {year}: {landsat_size}")
        if landsat_size > 0:
            image_list = landsat.toList(landsat_size)
            for i in range(landsat_size):
                image = ee.Image(image_list.get(i))
                spacecraft = image.get('SPACECRAFT_ID').getInfo()
                path = image.get('WRS_PATH').getInfo()
                row = image.get('WRS_ROW').getInfo()
                date_acquired = image.get('DATE_ACQUIRED').getInfo()
                image_id = image.get('system:id').getInfo()
                print(f"Image {i+1}: {spacecraft} - Path: {path} - Row: {row} - Date: {date_acquired} - ID: {image_id}")

            # Compute mean composite for average summer conditions
            landsat = landsat.mean()
            # Compute surface reflectance (divide by 10000 per Landsat scaling)
            reflectance = landsat.divide(10000)

            # NDVI formula: (NIR - Red) / (NIR + Red) using SR_B5 and SR_B4
            ndvi = reflectance.normalizedDifference(['SR_B5', 'SR_B4']).rename(f'NDVI_{year}').clip(geometry).unmask(0)
            ndvi_layers[year] = ndvi
            print(f"NDVI for {year} calculated successfully!")

            # PISI formula: 0.8192 * ρ_Blue - 0.5735 * ρ_NIR + 0.0750
            # Uses SR_B2 (Blue) and SR_B5 (NIR) as per provided equation
            blue = reflectance.select('SR_B2').rename('Blue')  # Blue band
            nir = reflectance.select('SR_B5').rename('NIR')    # Near-infrared band
            pisi = blue.multiply(0.8192).subtract(nir.multiply(0.5735)).add(0.0750).rename(f'PISI_{year}').clip(geometry).unmask(0)
            # No normalization needed as the equation inherently scales; verify range later
            pisi_layers[year] = pisi
            print(f"PISI for {year} calculated successfully!")
        else:
            print(f"No valid Landsat data for {year}")
            ndvi_layers[year] = None
            pisi_layers[year] = None
    return ndvi_layers, pisi_layers

In [5]:
# Add NDVI and PISI layers to the map
def add_layers(map_obj, city_boundary, ndvi_layers, pisi_layers):
    """
    NDVI: Blue (low vegetation) to green (high vegetation).
    PISI: Green (pervious) to brown (impervious surfaces).
    """
    geometry = city_boundary.geometry()
    ndvi_palette = ['blue', 'white', 'green']
    ndvi_vis = {"min": -1, "max": 1, "palette": ndvi_palette}
    pisi_palette = ['green', 'white', 'brown']
    pisi_vis = {"min": -1, "max": 1, "palette": pisi_palette}
    
    for year in years:
        if ndvi_layers.get(year) is not None:
            map_obj.addLayer(ndvi_layers[year], ndvi_vis, f'NDVI_{year}')
        if pisi_layers.get(year) is not None:
            map_obj.addLayer(pisi_layers[year], pisi_vis, f'PISI_{year}')
    
    return map_obj

In [6]:
# Export layers as GeoTIFFs to Google Drive for further analysis
def export_layers(ndvi_layers, pisi_layers, city_boundary):
    """
    Uses 30m scale and EPSG:20438 CRS for consistency with LST data.
    """
    geometry = city_boundary.geometry()
    for year in years:
        if ndvi_layers.get(year) is not None:
            ee.batch.Export.image.toDrive(
                image=ndvi_layers[year],
                description=f'Riyadh_NDVI_EPSG20438_updated{year}',
                folder='Riyadh_Layers',
                fileNamePrefix=f'Riyadh_NDVI_EPSG20438_updated{year}',
                region=geometry,
                scale=30, # Landsat resolution for NDVI
                crs='EPSG:20438', # UTM Zone 38N for metric accuracy as per https://epsg.io/20438
                maxPixels=1e9
            ).start()
            print(f"Exporting NDVI_{year} to Google Drive")
        if pisi_layers.get(year) is not None:
            ee.batch.Export.image.toDrive(
                image=pisi_layers[year],
                description=f'Riyadh_PISI_EPSG20438_{year}',
                folder='Riyadh_Layers',
                fileNamePrefix=f'Riyadh_PISI_EPSG20438_{year}',
                region=geometry,
                scale=30,
                crs='EPSG:20438',
                maxPixels=1e9
            ).start()
            print(f"Exporting PISI_{year} to Google Drive")

In [7]:
# Load and map
asset_id = 'projects/uss-dissertation/assets/riyadh_boundary'  # Link to asset: https://code.earthengine.google.com/?asset=projects/uss-dissertation/assets/riyadh_boundary
# Also, the city boundary used can be found in the  Data folder 'Data/Riyadh_boundary.geojson'
city_boundary = load_city_boundary(asset_id)
map_obj = geemap.Map(basemap='CartoDB.Positron')
boundary_style = {"color": "black", "fillColor": "00000000", "width": 2}
map_obj.addLayer(city_boundary, boundary_style, "Riyadh City Boundary")
map_obj.setCenter(46.75, 24.78, 12)

# Calculate and add layers
years = [2024]
ndvi_layers, pisi_layers = calculate_ndvi_pisi(city_boundary, years)
map_obj = add_layers(map_obj, city_boundary, ndvi_layers, pisi_layers)

# Export too google drive, 
export_layers(ndvi_layers, pisi_layers, city_boundary)

map_obj

City boundary loaded successfully!
Number of Landsat images for 2024: 32
Image 1: LANDSAT_8 - Path: 165 - Row: 43 - Date: 2024-06-01 - ID: LANDSAT/LC08/C02/T1_L2/LC08_165043_20240601
Image 2: LANDSAT_8 - Path: 165 - Row: 43 - Date: 2024-06-17 - ID: LANDSAT/LC08/C02/T1_L2/LC08_165043_20240617
Image 3: LANDSAT_8 - Path: 165 - Row: 43 - Date: 2024-07-03 - ID: LANDSAT/LC08/C02/T1_L2/LC08_165043_20240703
Image 4: LANDSAT_8 - Path: 165 - Row: 43 - Date: 2024-07-19 - ID: LANDSAT/LC08/C02/T1_L2/LC08_165043_20240719
Image 5: LANDSAT_8 - Path: 165 - Row: 43 - Date: 2024-08-04 - ID: LANDSAT/LC08/C02/T1_L2/LC08_165043_20240804
Image 6: LANDSAT_8 - Path: 165 - Row: 43 - Date: 2024-08-20 - ID: LANDSAT/LC08/C02/T1_L2/LC08_165043_20240820
Image 7: LANDSAT_8 - Path: 166 - Row: 42 - Date: 2024-06-08 - ID: LANDSAT/LC08/C02/T1_L2/LC08_166042_20240608
Image 8: LANDSAT_8 - Path: 166 - Row: 42 - Date: 2024-06-24 - ID: LANDSAT/LC08/C02/T1_L2/LC08_166042_20240624
Image 9: LANDSAT_8 - Path: 166 - Row: 42 - Date

Map(center=[24.78, 46.75], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…